In [1]:
import psycopg2
from tqdm.auto import tqdm
from typing import Dict

import numpy as np
import pandas as pd


In [2]:
conn = psycopg2.connect("dbname=patents user=martin host=localhost password=F0hn1lcob.r")

In [3]:
cur = conn.cursor('code_download')

In [ ]:
cur.execute("SELECT publication_number, code from patents, unnest()")

In [ ]:
with open("/var/patentmark/citation_edges_eu_jp_us.txt", "w") as outfile:
    for row in tqdm(cur):
        outfile.write(str(row[0])) 
        outfile.write(" ")
        outfile.write(' '.join(map(str, row[1])))
        outfile.write("\n")

In [4]:
cur.execute("select patent_id, publication_number from patents;")
with open("/var/patentmark/citation_mapping.csv", "w") as outfile:
    for row in tqdm(cur):
        outfile.write(str(row[0]))
        outfile.write(",")
        outfile.write(row[1])
        outfile.write("\n")
        

In [ ]:
np_embeddings = np.fromfile("/var/patentmark/citations.verse.32d", dtype=np.float32)

In [ ]:
import pandas as pd
id_lookup = pd.read_csv("/var/patentmark/citation_mapping.csv", header=None)

In [ ]:
id_lookup.columns = ["patent_id", "publication_number"]

In [ ]:
id_lookup.set_index("patent_id", inplace=True)

In [ ]:
id_lookup

In [ ]:
np_embeddings = np_embeddings.reshape(-1,32)

In [ ]:
embedding_frame = pd.DataFrame(np_embeddings).apply(np.array, axis=1)

In [ ]:
embedding_frame.shape

In [ ]:
embedding_frame.columns = ["citation_based_embedding"]

In [ ]:
embeddings = embedding_frame.to_frame().join(id_lookup, how="inner")

In [ ]:
embeddings.set_index("publication_number", inplace=True)

In [ ]:
embeddings.columns=["citation_based_embedding"]

In [ ]:
embeddings

In [ ]:
embeddings.to_parquet("citation_embeddings.parquet")